In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import random as rnd
import math
from pytrends.request import TrendReq
from seaborn import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, fbeta_score, classification_report

In [9]:
pytrends = TrendReq(hl='en-US', tz=360)

kw_list = ["Tesla"] 

pytrends.build_payload(kw_list, timeframe='2021-03-30 2021-06-29', geo='US')

Trendsdf = pytrends.interest_over_time()
Trendsdf = Trendsdf.rename(columns={'Tesla': 'Previous_Search_Interest'})
#Trendsdf.reset_index(inplace=True, drop=True)
Trendsdf.shape
Trendsdf

,Previous_Search_Interest,isPartial
date,,
2021-03-30,81,False
2021-03-31,71,False
2021-04-01,78,False
2021-04-02,72,False
2021-04-03,65,False
...,...,...
2021-06-25,71,False
2021-06-26,69,False
2021-06-27,65,False


In [34]:
stockPricePrevious = yf.download("TSLA", start="2021-03-31", end="2021-06-29")
stockPricePrevious = stockPricePrevious.rename(columns={'Open': 'Previous_Open', 'Close': 'Previous_Close','High': 'Previous_High','Volume': 'Previous_Volume','Low': 'Previous_Low', 'Adj Close':'Previous_Adj_Close'})
stockPricePrevious.reset_index(inplace=True, drop=True)
stockPricePrevious
#print(stockPricePrevious)
stockPrice = yf.download("TSLA", start="2021-04-01", end="2021-06-30")
stockPrice.reset_index(inplace=True, drop=True)
#print(stockPrice)
mergedPrice = pd.concat([stockPricePrevious, stockPrice], axis=1)
#mergedPrice['Previous_Open', 'Previous_High','Previous_Low','Previous_Close', 'Previous_Adj_Close', 'Previous_Volume'] = mergedPrice['Previous_Open', 'Previous_High','Previous_Low','Previous_Close', 'Previous_Adj_Close', 'Previous_Volume'].shift(-1)
#mergedPrice.dropna(inplace=True)
mergedPrice

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Adj_Close,Previous_Volume,Open,High,Low,Close,Adj Close,Volume
0,646.619995,672.000000,641.109985,667.929993,667.929993,33337300,688.369995,692.419983,659.419983,661.750000,661.750000,35298400
1,688.369995,692.419983,659.419983,661.750000,661.750000,35298400,707.710022,708.159973,684.700012,691.049988,691.049988,41842800
2,707.710022,708.159973,684.700012,691.049988,691.049988,41842800,690.299988,696.549988,681.369995,691.619995,691.619995,28271800
3,690.299988,696.549988,681.369995,691.619995,691.619995,28271800,687.000000,691.380005,667.840027,670.969971,670.969971,26309400
4,687.000000,691.380005,667.840027,670.969971,670.969971,26309400,677.380005,689.549988,671.650024,683.799988,683.799988,23924300
...,...,...,...,...,...,...,...,...,...,...,...,...
57,618.250000,628.570007,615.500000,623.710022,623.710022,19158900,632.000000,657.200012,630.039978,656.570007,656.570007,31099200
58,632.000000,657.200012,630.039978,656.570007,656.570007,31099200,674.989990,697.619995,667.609985,679.820007,679.820007,45982400
59,674.989990,697.619995,667.609985,679.820007,679.820007,45982400,689.580017,693.809998,668.700012,671.869995,671.869995,32496700
60,689.580017,693.809998,668.700012,671.869995,671.869995,32496700,671.640015,694.700012,670.320007,688.719971,688.719971,21628200
